In [ ]:
from sentence_transformers import SentenceTransformer
import pandas as pd
import numpy as np

# Load your cleaned data (CSV with 'text' column)
df = pd.read_csv("/Users/gwin/Documents/Post Undergrad Work/Tax Search/test_data/junk/gov_docs.csv")  

# Use tiny-but-mighty model (CPU-friendly)
model = SentenceTransformer("all-MiniLM-L6-v2")  # 384-dim vectors


.gitattributes: 100%|██████████| 1.23k/1.23k [00:00<00:00, 215kB/s]
config.json: 100%|██████████| 190/190 [00:00<00:00, 101kB/s]
README.md: 100%|██████████| 10.5k/10.5k [00:00<00:00, 6.59MB/s]
config_sentence_transformers.json: 100%|██████████| 116/116 [00:00<00:00, 73.2kB/s]
data_config.json: 100%|██████████| 39.3k/39.3k [00:00<00:00, 7.09MB/s]
model.safetensors: 100%|██████████| 90.9M/90.9M [00:07<00:00, 12.9MB/s]
model.onnx: 100%|██████████| 90.4M/90.4M [00:09<00:00, 9.62MB/s]
model_O1.onnx: 100%|██████████| 90.4M/90.4M [00:06<00:00, 13.2MB/s]
model_O2.onnx: 100%|██████████| 90.3M/90.3M [00:07<00:00, 12.5MB/s]
model_O3.onnx: 100%|██████████| 90.3M/90.3M [00:07<00:00, 11.7MB/s]
model_O4.onnx: 100%|██████████| 45.2M/45.2M [00:03<00:00, 13.0MB/s]
model_qint8_arm64.onnx: 100%|██████████| 23.0M/23.0M [00:01<00:00, 12.8MB/s]
model_qint8_arm64.onnx: 100%|██████████| 23.0M/23.0M [00:01<00:00, 13.4MB/s]
model_qint8_arm64.onnx: 100%|██████████| 23.0M/23.0M [00:01<00:00, 13.1MB/s]
model_quint8

TypeError: 'float' object is not subscriptable

In [ ]:
# Drop rows with missing text
df = df.dropna(subset=['text']) 

# Or fill empty values with empty string
df['text'] = df['text'].fillna('')
df["vector"] = df["text"].apply(lambda x: model.encode(x).tolist())

In [10]:
from elasticsearch import Elasticsearch

# Start local Elasticsearch (Docker)
# docker run -p 9200:9200 -e "discovery.type=single-node" elasticsearch:8.12.0

es = Elasticsearch("http://localhost:9200")

# Create index
es.indices.create(index="docs", mappings={
    "properties": {
        "text": {"type": "text"},
        "vector": {
            "type": "dense_vector",
            "dims": 384,
            "index": True,  # Enable HNSW for fast search
            "similarity": "cosine"
        }
    }
})

# Insert data
for _, row in df.iterrows():
    es.index(index="docs", document={
        "text": row["text"],
        "vector": row["vector"]
    })

In [ ]:
query = "got students to take care of "
query_vector = model.encode(query).tolist()

results = es.search(index="docs", knn={
    "field": "vector",
    "query_vector": query_vector,
    "k": 10,
    "num_candidates": 100
})

for hit in results["hits"]["hits"]:
    print(f"Score: {hit['_score']:.3f}\nText: {hit['_source']['text'][:200]}...\n")

Score: 0.613
Text: OFFICEOFTHEINSPECTORGENERALI ~~uJ>ll~~~ Department of Defense...

Score: 0.612
Text: pg. 1 www.fda.gov Proposed Regulatory Framework for Modifications to Artificial Intelligence/Machine Learning (AI/ML)-Based Software as a Medical Device (SaMD) - Discussion Paper and Request for Feedb...

Score: 0.604
Text: 1 NEWYORKSTATEEDUCATIONDEPARTMENTPBOLICYRIEFFormula Allocation For Schools: Historical Perspective and Lessons from New York State PREPAREDFORTHENATIONALACADEMYFORSCIENCESPANELONFORMULAALLOCATIONBYJAM...

Score: 0.598
Text: Current Research Resources in DEFENSEACQUISITIONACQUISITIONREFORMEach issue of the Defense Acquisition Research Journal will bring to the attention of the defense acquisition community a topic of curr...

Score: 0.596
Text: THEDoDOFTOMORROW: Innovating American Defense contents 03 Executive Summary | 04 The Changing Mission of DoD 06 14 18 26 Operations The Power of Technology Workforce Data at DoD • Consolidating and • ...

Score: 0.591
Text: 